# Assignment 2

Deadline: 26.03.2025, 12:00 CET

Weijie Chen, 23-743-727, weijie.chen@uzh.ch

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [10]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))  # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.quadratic_program import QuadraticProgram
from helper_functions import load_data_msci

In [127]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
        '''
        Linearize the turnover constraint in the quadratic programming problem.

        This method modifies the quadratic programming problem to include a linearized turnover constraint.

        Parameters:
        -----------
        x_init : np.ndarray
            The initial portfolio weights.
        to_budget : float, optional
            The maximum allowable turnover. Defaults to float('inf').

        Notes:
        ------
        - The method updates the problem's objective function coefficients, inequality constraints,
        equality constraints, and bounds to account for the turnover constraint.
        - The original problem data is overridden with the updated matrices and vectors.

        Examples:
        ---------
        >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
        >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
        '''
        P = self.problem_data.get('P')
        q = self.problem_data.get('q')
        A = self.problem_data.get('A')
        G = self.problem_data.get('G')
        h = self.problem_data.get('h')
        # print("G",G) -> None
        # print("h",h) -> None
        # print("A",A) -> A [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
        # print("b",b) -> b 1.0

        # Dimensions
        n = len(q)
        m = 0 if G is None else G.shape[0]
        
        # Update the coefficients of the objective function
        P_new = np.pad(P, ((0, n), (0, n)), 'constant') # pad P with zeros
        q_new = np.pad(q, (0, n), 'constant') # pad q with zeros
        
        
        # Update the equality constraints A
        if A is not None:
            A_new = np.pad(A, ((0, 0), (0, n)), 'constant')
        else:
            A_new = None
        
        # Update the inequality constraints G and h
        I_n = np.eye(n)
        zero_n = np.zeros((n, n))
      
        if G is None:
            G = np.zeros((0, n))
        # print("G_old.shape after:", G_old.shape)
        # print("np.zeros((G_old.shape[0], n)).shape:", np.zeros((G_old.shape[0], n)).shape)
        
        if h is None:
            h = np.zeros(0)

        turnover_G = np.vstack([
            np.hstack([I_n, -I_n]),  # x - x_init <= z
            np.hstack([-I_n, -I_n])  # x_init - x <= z
        ])

        turnover_h = np.full(n * 2, to_budget / n)

    
        G_new = np.vstack([
            np.hstack([G, np.zeros((m, n))]), 
            turnover_G 
        ])

        h_new = np.hstack([h, turnover_h])

        # Update lower and upper bounds
        lb_new = np.pad(self.problem_data['lb'], (0, n), 'constant')  # w >= 0
        ub_new = np.pad(self.problem_data['ub'], (0, n), 'constant', constant_values=1e6)  

        # Override the original matrices (notice: b does not change)
        self.update_problem_data({
            'P': P_new,
            'q': q_new,
            'G': G_new,
            'h': h_new,
            'A': A_new,
            'lb': lb_new,
            'ub': ub_new
        })

        return None

## Demo

#### Create P and q

In [11]:
# Load the msci country index data
N = 10
data = load_data_msci(path = '../data/', n = N)
X = data['return_series']

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method='geometric').estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method='pearson').estimate(X=X, inplace=False)

# q, P
print(q)
print(P)

AT    0.000130
AU    0.000288
BE    0.000047
CA    0.000269
CH    0.000149
DE    0.000151
DK    0.000429
ES    0.000128
FI    0.000145
FR    0.000199
dtype: float64
          AT        AU        BE        CA        CH        DE        DK  \
AT  0.000239  0.000054  0.000125  0.000075  0.000097  0.000138  0.000097   
AU  0.000054  0.000104  0.000039  0.000030  0.000035  0.000041  0.000041   
BE  0.000125  0.000039  0.000175  0.000064  0.000104  0.000137  0.000093   
CA  0.000075  0.000030  0.000064  0.000130  0.000058  0.000087  0.000053   
CH  0.000097  0.000035  0.000104  0.000058  0.000120  0.000121  0.000086   
DE  0.000138  0.000041  0.000137  0.000087  0.000121  0.000202  0.000105   
DK  0.000097  0.000041  0.000093  0.000053  0.000086  0.000105  0.000151   
ES  0.000150  0.000044  0.000138  0.000081  0.000116  0.000164  0.000100   
FI  0.000140  0.000050  0.000136  0.000091  0.000126  0.000180  0.000119   
FR  0.000143  0.000045  0.000142  0.000084  0.000122  0.000174  0.000107   

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [131]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids = X.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0,
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Solve the problem
qp.solve()

# Access and print the solver status
solver_status = qp.results.get('status')

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)


### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [132]:

# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=0.5)

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get('solution')
ids = constraints.ids
weights = pd.Series(solution.x[:len(ids)], index=ids)

# print("Weights:", x_init)
# print("Solution:", weights)
print("Turnover:")
print(np.abs(weights - x_init).sum())
# print("Max violation of turnover constraint:", np.max(np.abs(weights - x_init)) - 0.5)
# print("Total Turnover:", np.abs(weights - x_init).sum())




Turnover:
1.2008719147159819
